In [2]:
import xgboost as xgb
import csv
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [12]:
def readtrain():
    with open('train_modified.csv', 'rb') as csvfile:
        reader = csv.reader(csvfile)
        column1 = [row for row in reader]
    content_train = [i[4] for i in column1[1:]] 
    opinion_train = [i[0] for i in column1[1:]] 
    print '有 %s 条' % len(content_train)
    train = [content_train, opinion_train]
    return train

train = readtrain()

有 739 条


In [4]:
def get_ngrams(query):
    ngrams = []
    text = nltk.word_tokenize(str(query))
    for w in text:
    	w=w.lower().lstrip(' ').rstrip(' ').replace("u'",' ')
    	if w[0]=="'":
    		w=w[1:-1]
    	w=w.lstrip(' ').rstrip(' ')
    	ngrams.append(w)
    return ngrams

In [5]:
def segmentWord(cont):
    c = []
    for i in cont:
        a = get_ngrams(i)
        b = " ".join(a)
        c.append(b)
    return c
content = segmentWord(train[0])


In [6]:
def transLabel(labels):
    for i in range(len(labels)):
        if labels[i] == 'n':
            labels[i] = 1
        else:
            labels[i]=0
    return labels
opinion = transLabel(train[1]) 
opinion = np.array(opinion) 


In [7]:
train_content = content[:550]
train_opinion = opinion[:550]
test_content = content[550:]
test_opinion = opinion[550:]

In [8]:
vectorizer = CountVectorizer()
tfidftransformer = TfidfTransformer()
tfidf = tfidftransformer.fit_transform(vectorizer.fit_transform(train_content))
weight = tfidf.toarray()
print tfidf.shape
test_tfidf = tfidftransformer.transform(vectorizer.transform(test_content))
test_weight = test_tfidf.toarray()
print test_weight.shape

(550, 3119)
(189L, 3119L)


In [11]:
dtrain = xgb.DMatrix(weight, label=train_opinion)
dtest = xgb.DMatrix(test_weight, label=test_opinion)
param = {'max_depth':7, 'eta':0.1, 'eval_metric':'merror', 'silent':1, 'objective':'multi:softmax', 'num_class':2}  
evallist  = [(dtrain,'train'), (dtest,'test')]  
num_round = 52  
bst = xgb.train(param, dtrain, num_round, evallist)
preds = bst.predict(dtest)

[0]	train-merror:0.054545	test-merror:0.148148
[1]	train-merror:0.054545	test-merror:0.148148
[2]	train-merror:0.054545	test-merror:0.148148
[3]	train-merror:0.054545	test-merror:0.148148
[4]	train-merror:0.054545	test-merror:0.148148
[5]	train-merror:0.054545	test-merror:0.148148
[6]	train-merror:0.054545	test-merror:0.148148
[7]	train-merror:0.054545	test-merror:0.148148
[8]	train-merror:0.054545	test-merror:0.148148
[9]	train-merror:0.054545	test-merror:0.148148
[10]	train-merror:0.054545	test-merror:0.148148
[11]	train-merror:0.054545	test-merror:0.148148
[12]	train-merror:0.054545	test-merror:0.148148
[13]	train-merror:0.054545	test-merror:0.148148
[14]	train-merror:0.054545	test-merror:0.148148
[15]	train-merror:0.054545	test-merror:0.148148
[16]	train-merror:0.054545	test-merror:0.089947
[17]	train-merror:0.054545	test-merror:0.148148
[18]	train-merror:0.054545	test-merror:0.148148
[19]	train-merror:0.050909	test-merror:0.089947
[20]	train-merror:0.054545	test-merror:0.089947
[2